In [233]:
# 📚 Librerías Necesarias

# 🔄 Preprocesamiento de Datos
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 📊 Métricas de Evaluación
from sklearn.metrics import mean_squared_error, r2_score

# 🧠 Modelado con TensorFlow y Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 💾 Guardado y Carga de Modelos
import joblib

# 📈 Visualización
import matplotlib.pyplot as plt

# ⚙️ Sistema Operativo
import os

In [234]:
# ✅ Parámetros Globales optimizados para CPU
IMG_SIZE = (128, 128)  # Tamaño de las imágenes para el modelo
BATCH_SIZE = 16        # Tamaño del lote para evitar sobrecarga en CPU
EPOCHS = 30            # Número de épocas (puedes ajustarlo según los resultados)
COLOR_MODE = 'rgb'     # Imágenes en color (TensorFlow espera RGB, OpenCV carga BGR)
AUTOTUNE = tf.data.AUTOTUNE  # Optimización automática para la carga de datos


In [235]:
# ✅ Rutas de Datos
ruta_train_csv = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train.csv'
ruta_test_csv = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test.csv'
ruta_train_img = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train'
ruta_test_img = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test'

# ✅ Cargar Datos
df_train = pd.read_csv(ruta_train_csv)
df_test = pd.read_csv(ruta_test_csv)

In [236]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      5000 non-null   int64
dtypes: int64(1)
memory usage: 39.2 KB


In [237]:
df_test.describe()

,id
count,5000.000000
mean,47500.500000
std,1443.520003
min,45001.000000
25%,46250.750000
50%,47500.500000
75%,48750.250000
max,50000.000000


In [238]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   id          15000 non-null  int64
 1   clip_count  15000 non-null  int64
dtypes: int64(2)
memory usage: 234.5 KB


In [239]:
df_train.describe()

,id,clip_count
count,15000.000000,15000.000000
mean,37500.500000,37.290133
std,4330.271354,21.922691
min,30001.000000,0.000000
25%,33750.750000,18.000000
50%,37500.500000,37.000000
75%,41250.250000,56.000000
max,45000.000000,75.000000


In [240]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

# ✅ Función para ajustar brillo, contraste y saturación
def ajustar_imagen_opencv(img, brillo=0, contraste=0, saturacion=0):
    """
    Ajusta el brillo, contraste y saturación de una imagen en color.
    Args:
        img: Imagen en formato OpenCV (BGR).
        brillo: Valor para ajustar el brillo (-100 a 100).
        contraste: Valor para ajustar el contraste (-100 a 100).
        saturacion: Valor para ajustar la saturación (-100 a 100).
    Returns:
        Imagen ajustada.
    """
    # Ajuste de Brillo y Contraste
    img = cv2.convertScaleAbs(img, alpha=1 + (contraste / 100), beta=brillo)
    
    # Ajuste de Saturación (en espacio HSV)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_hsv = img_hsv.astype(np.float32)
    img_hsv[:, :, 1] = np.clip(img_hsv[:, :, 1] * (1 + saturacion / 100), 0, 255)
    img_hsv = img_hsv.astype(np.uint8)
    img = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)
    
    return img

# ✅ Mostrar ajuste interactivo
def ajuste_interactivo(ruta_img, img_id):
    """
    Ajusta brillo, contraste y saturación interactivamente usando una imagen específica.
    Args:
        ruta_img: Ruta de la carpeta de imágenes.
        img_id: ID de la imagen.
    """
    img_path = os.path.join(ruta_img, f"clips-{img_id}.png")
    
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"No se pudo encontrar la imagen en {img_path}")
    
    img = cv2.imread(img_path)  # Cargar imagen en color (BGR)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen en {img_path}")
    
    def actualizar(brillo=0, contraste=0, saturacion=0):
        img_ajustada = ajustar_imagen_opencv(img, brillo, contraste, saturacion)
        
        # Mostrar solo dos imágenes una al lado de la otra
        plt.figure(figsize=(14, 7))
        
        # Imagen Original
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title('Imagen Original')
        plt.axis('off')
        
        # Imagen Ajustada
        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(img_ajustada, cv2.COLOR_BGR2RGB))
        plt.title(f'Ajustada (Brillo: {brillo}, Contraste: {contraste}, Saturación: {saturacion})')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    # Llamar a la función interactiva
    interact(
        actualizar,
        brillo=(-100, 100, 5),
        contraste=(-100, 100, 5),
        saturacion=(-100, 100, 5)
    )

# ✅ Llamar a la función con una imagen específica
ruta_train_img = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train'
ajuste_interactivo(ruta_train_img, img_id='30002')


interactive(children=(IntSlider(value=0, description='brillo', min=-100, step=5), IntSlider(value=0, descripti…

In [241]:
import tensorflow as tf
import cv2
import numpy as np

IMG_SIZE = (128, 128)  # Tamaño de la imagen
BATCH_SIZE = 16        # Tamaño del lote
AUTOTUNE = tf.data.AUTOTUNE

# ✅ Función para ajustar brillo, contraste y saturación
def ajustar_imagen_tf(img, brillo=-10, contraste=5, saturacion=-100):
    """
    Aplica ajustes de brillo, contraste y saturación a la imagen usando TensorFlow.
    """
    # Ajuste de Brillo
    img = tf.image.adjust_brightness(img, delta=brillo)
    
    # Ajuste de Contraste
    img = tf.image.adjust_contrast(img, contrast_factor=1 + (contraste / 100))
    
    # Ajuste de Saturación (ajuste de matiz en lugar de saturación exacta)
    img = tf.image.adjust_hue(img, delta=saturacion * 0.01)  # Saturación controlada por matiz
    
    return img

# ✅ Preprocesamiento de imágenes optimizado para TensorFlow
def cargar_datos(df, ruta_img, es_entrenamiento=True):
    """
    Esta función carga y preprocesa las imágenes desde la ruta indicada,
    aplicando los ajustes de brillo, contraste y saturación.
    """
    ids = df['id'].values
    labels = df['clip_count'].values if 'clip_count' in df.columns else None

    def preprocesar_imagen(id, label=None):
        # Cargar la imagen desde la ruta
        img_path = tf.strings.join([ruta_img, "/clips-", tf.strings.as_string(id), ".png"])
        img = tf.io.read_file(img_path)
        img = tf.image.decode_png(img, channels=3)  # Asegurarse de cargar imágenes en RGB (3 canales)
        img = tf.image.resize(img, IMG_SIZE)  # Cambiar el tamaño de la imagen
        img = img / 255.0  # Normalizar la imagen

        # Aplicar ajustes de brillo, contraste y saturación usando TensorFlow
        img = ajustar_imagen_tf(img, brillo=-10, contraste=5, saturacion=-100)

        return (img, tf.cast(label, tf.float32)) if label is not None else img

    # Crear el dataset
    dataset = tf.data.Dataset.from_tensor_slices((ids, labels) if es_entrenamiento else ids)
    dataset = dataset.map(
        lambda id, label=None: preprocesar_imagen(id, label),
        num_parallel_calls=AUTOTUNE
    )
    dataset = dataset.cache()

    # Solo en entrenamiento realizamos shuffle
    if es_entrenamiento:
        dataset = dataset.shuffle(buffer_size=500)
    
    # Crear batches y optimizar el rendimiento
    dataset = dataset.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

    return dataset



In [242]:
# ✅ Crear Datasets
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)
dataset_train = cargar_datos(df_train, ruta_train_img, es_entrenamiento=True)
dataset_val = cargar_datos(df_val, ruta_train_img, es_entrenamiento=True)
dataset_test = cargar_datos(df_test, ruta_test_img, es_entrenamiento=False)


In [243]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import tensorflow as tf

# ✅ Definir el Modelo CNN optimizado para CPU
def crear_modelo():
    modelo = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),  # RGB, 3 canales
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='linear')  # Salida de regresión
    ])

    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error',
        metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')]
    )
    return modelo

# ✅ Crear el modelo
modelo = crear_modelo()

# ✅ Ver el resumen del modelo
modelo.summary()



C:\Users\mikel\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_21 (Conv2D)              │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,665 (12.61 MB)

 Trainable params: 3,305,217 (12.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [244]:
# ✅ Callbacks optimizados
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=3)

# ✅ Entrenar Modelo
historial = modelo.fit(
    dataset_train,
    validation_data=dataset_val,
    epochs=EPOCHS,
    callbacks=[early_stop, reduce_lr]
)

# ✅ Graficar Métricas
plt.plot(historial.history['rmse'], label='RMSE (Train)')
plt.plot(historial.history['val_rmse'], label='RMSE (Validation)')
plt.legend()
plt.show()

Epoch 1/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 152s 200ms/step - loss: 189.4321 - rmse: 13.6157 - val_loss: 23713.2715 - val_rmse: 153.9911 - learning_rate: 0.0010
Epoch 2/30
464/750 ━━━━━━━━━━━━━━━━━━━━ 1:07 236ms/step - loss: 118.9433 - rmse: 10.9023

KeyboardInterrupt: 

In [53]:
# ✅ Evaluación
train_loss, train_rmse = modelo.evaluate(dataset_train)
val_loss, val_rmse = modelo.evaluate(dataset_val)
print(f"🔹 RMSE (Train): {train_rmse:.2f}")
print(f"🔹 RMSE (Validation): {val_rmse:.2f}")

# ✅ Predicciones
predicciones = modelo.predict(dataset_test).flatten()
df_test['clip_count'] = np.round(predicciones).astype(int)

# ✅ Guardar Resultados
df_test[['id', 'clip_count']].to_csv('submission.csv', index=False)
print("✅ Archivo 'submission.csv' generado correctamente.")

750/750 ━━━━━━━━━━━━━━━━━━━━ 52s 70ms/step - loss: 3.5887 - rmse: 1.8941
188/188 ━━━━━━━━━━━━━━━━━━━━ 13s 70ms/step - loss: 4.5850 - rmse: 2.1409
🔹 RMSE (Train): 1.88
🔹 RMSE (Validation): 2.12
313/313 ━━━━━━━━━━━━━━━━━━━━ 30s 97ms/step
✅ Archivo 'submission_p.csv' generado correctamente.


In [54]:
# ✅ Guardar el Modelo en la Carpeta 'modelo_final'
ruta_modelo_completo = os.path.join('modelo_final', 'modelo_clip_count_p.h5')
modelo.save(ruta_modelo_completo)
print(f"✅ Modelo guardado en '{ruta_modelo_completo}'")

✅ Modelo guardado en 'modelo_final\modelo_clip_count_p.h5'
